<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#imports" data-toc-modified-id="imports-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>imports</a></span></li><li><span><a href="#methods" data-toc-modified-id="methods-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>methods</a></span></li></ul></li><li><span><a href="#Planning-new-network" data-toc-modified-id="Planning-new-network-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Planning new network</a></span></li><li><span><a href="#go-through-the-steps-to-make-sure-i-remember" data-toc-modified-id="go-through-the-steps-to-make-sure-i-remember-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>go through the steps to make sure i remember</a></span><ul class="toc-item"><li><span><a href="#re-organize-data" data-toc-modified-id="re-organize-data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>re-organize data</a></span></li><li><span><a href="#get_pairs()" data-toc-modified-id="get_pairs()-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>get_pairs()</a></span></li><li><span><a href="#network-building" data-toc-modified-id="network-building-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>network building</a></span></li><li><span><a href="#train-network" data-toc-modified-id="train-network-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>train network</a></span></li><li><span><a href="#initial-run-complete" data-toc-modified-id="initial-run-complete-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>initial run complete</a></span></li><li><span><a href="#exorcism" data-toc-modified-id="exorcism-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>exorcism</a></span></li></ul></li></ul></div>

## imports

In [32]:
import keras
import numpy as np
import datetime

from keras.preprocessing.image import ImageDataGenerator
from collections import defaultdict
from glob import glob
from random import choice, sample

# Florencia's imports
import tensorflow
from tensorflow.keras import layers, preprocessing, models, optimizers
from tensorflow.keras.layers import Input, Conv2D, Lambda, Dense, Flatten,MaxPooling2D,Activation, Dropout, BatchNormalization,  GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Subtract,Multiply
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from skimage.io import imshow
import matplotlib.pyplot as plt
import numpy as np
from numpy import array
import pandas as pd
import random
import cv2
import os
import pickle
from os import listdir
from tqdm import tqdm
from tensorflow.keras.applications.resnet50 import preprocess_input
from random import sample, choice

from tensorflow.keras import backend as K 
from keras.preprocessing.image import ImageDataGenerator 
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.backend import clear_session
from random import randint
from random import uniform as rfloat
import h5py
from tensorflow.keras.models import model_from_json 
from keras import regularizers
import os 
from tensorflow.keras.optimizers import Adam


## methods

In [92]:
imgs[0]

'/Users/mark/Documents/flatiron/projects/facial_similarity/siamese_similarity_cnn/convolutional_siamese_facial_similarity_keras/UTKface/backup/26_1_1_20170103181931657.jpg.chip.jpg'

In [103]:
jpg = read_img(imgs[0])

In [104]:
jpg.shape

(200, 200, 3)

In [ ]:
    img = cv2.resize(img, (200,200))


In [106]:
jpg = cv2.resize(jpg,(200,200))
jpg.shape

(200, 200, 3)

In [109]:
def read_img(path):
    img = cv2.imread(path) # don't put 0 as the second argument, that makes it ignore RGB channels
    img = cv2.resize(img,(200,200))
    img = np.array(img).astype(np.float)
    return preprocess_input(img)
def paths_to_arrays(paths_list):
    out = []
    for path in paths_list:
        out.append(read_img(path))
    return out

def get_img_paths(): 
    direc = os.getcwd()
    path = '/UTKface/backup/'
    imgpaths = glob(direc+path+'*.jpg')
    return imgpaths

def tvt_split(imgs): # 60, 20, 20
    train, val, test = np.split(imgs.sample(frac=1), [int(.6*len(imgs)), int(.8*len(imgs))])
    return train, val, test

def jpgs_to_arrays(trains, tests, vals):
    train_data = paths_to_arrays(trains)
    test_data = paths_to_arrays(tests)
    val_data = paths_to_arrays(vals)
    return train_data, test_data, val_data

def euclidean_distance(vects): # Euc_dist between 2 vectors
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))
def eucl_dist_output_shape(shapes): 
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred): # really cool stuff
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf'''
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

def random_aug(img, img_gen):
    transform_parameters = { # full transform params list @ keras image preprocessing documentation
         'shear':randint(-8,8),
        'zx':rfloat(0.9,1.0), "zy":rfloat(0.9,1.0), 
        'theta':randint(-20,20), "flip_horizontal":bool(random.getrandbits(1))}
#     img_gen = ImageDataGenerator()
    return img_gen.apply_transform(img, transform_parameters)

# Planning new network
- 2 limiting factors from last model:
    - data
        - 2,000 images is good, but i don't think augmentation will lead to great learning
        - use something like AT&Tfaces, which has ~40 'users' but 8 photos of each user
            - this is few-shot learning after all, the simulated extra shots may not be giving me the desired output
            - then again, it could also be:
    - architecture size
        - 6 layers, mixture of dense+conv+flattening. messed around with different activations
        - i think i should try changing this first, because it's something i want to learn for future models as well
- Model updates:
    - transfer learning
        - try importing VGGnet, Facenet etc and running
        - learn Sagemaker to run it so i'm not limited by local computing power


# go through the steps to make sure i remember

## re-organize data

## get_pairs()
core function is get_pairs().
the net intakes 2 images and outputs 1 or 0 for same or different label.
so our x is a list of [img_array_a, img_array_b] pairs, and our y is a list of [1, 0, 1] matching each pair

In [3]:
def get_pairs(imgs_list): # creates 50/50 split of 'same' and 'dif' image pairs & labels. len=n*5
    pairs = [] # x
    labels = [] # y
    print(f'making pairs from {len(imgs_list)} images...')
    for c, img in enumerate(imgs_list):
        temp = imgs_list # shallow copy messed this up
        for i in range(0,4): # make 4 'same' and 'dif' pairs for each base_image
            # same pair
            pairs.append([img, img])
            labels.append(1) # true
            # dif pair
            print(f'getting rand_ind between 0 and {len(temp)}')
            indx = randint(0,len(temp)-1)
            while indx==c: # if random index matches current iteration index
                indx = randint(0,len(temp)-1) # avoid deepcopy memory time
            print(f'index: {indx}')
            pairs.append([img, temp[indx]]) # make a "dif" pair of images
            labels.append(0)
            print(f'total {len(pairs)}')
        print(f'done with image #{c} \n')
    return pairs, labels # returns lists for x, y. pairs+labels matched up by index

- process
    - get all images in one big list
        - train_test_split into

In [9]:
# get all image paths
imgs = get_img_paths()
len(imgs)

2253

In [110]:
ars = paths_to_arrays(imgs)

In [111]:
# make x, y pairs+labels from images
x, y = get_pairs(ars)

making pairs from 2253 images...
getting rand_ind between 0 and 2253
index: 547
total 2
getting rand_ind between 0 and 2253
index: 1195
total 4
getting rand_ind between 0 and 2253
index: 20
total 6
getting rand_ind between 0 and 2253
index: 724
total 8
done with image #0 

getting rand_ind between 0 and 2253
index: 428
total 10
getting rand_ind between 0 and 2253
index: 691
total 12
getting rand_ind between 0 and 2253
index: 1248
total 14
getting rand_ind between 0 and 2253
index: 1251
total 16
done with image #1 

getting rand_ind between 0 and 2253
index: 1389
total 18
getting rand_ind between 0 and 2253
index: 1834
total 20
getting rand_ind between 0 and 2253
index: 752
total 22
getting rand_ind between 0 and 2253
index: 852
total 24
done with image #2 

getting rand_ind between 0 and 2253
index: 842
total 26
getting rand_ind between 0 and 2253
index: 61
total 28
getting rand_ind between 0 and 2253
index: 1087
total 30
getting rand_ind between 0 and 2253
index: 1216
total 32
done wi

getting rand_ind between 0 and 2253
index: 1811
total 2386
getting rand_ind between 0 and 2253
index: 1872
total 2388
getting rand_ind between 0 and 2253
index: 1085
total 2390
getting rand_ind between 0 and 2253
index: 1229
total 2392
done with image #298 

getting rand_ind between 0 and 2253
index: 371
total 2394
getting rand_ind between 0 and 2253
index: 2108
total 2396
getting rand_ind between 0 and 2253
index: 1754
total 2398
getting rand_ind between 0 and 2253
index: 2223
total 2400
done with image #299 

getting rand_ind between 0 and 2253
index: 1733
total 2402
getting rand_ind between 0 and 2253
index: 1362
total 2404
getting rand_ind between 0 and 2253
index: 84
total 2406
getting rand_ind between 0 and 2253
index: 544
total 2408
done with image #300 

getting rand_ind between 0 and 2253
index: 2036
total 2410
getting rand_ind between 0 and 2253
index: 749
total 2412
getting rand_ind between 0 and 2253
index: 550
total 2414
getting rand_ind between 0 and 2253
index: 267
total


getting rand_ind between 0 and 2253
index: 1290
total 4538
getting rand_ind between 0 and 2253
index: 2086
total 4540
getting rand_ind between 0 and 2253
index: 1930
total 4542
getting rand_ind between 0 and 2253
index: 1191
total 4544
done with image #567 

getting rand_ind between 0 and 2253
index: 1438
total 4546
getting rand_ind between 0 and 2253
index: 197
total 4548
getting rand_ind between 0 and 2253
index: 606
total 4550
getting rand_ind between 0 and 2253
index: 43
total 4552
done with image #568 

getting rand_ind between 0 and 2253
index: 1564
total 4554
getting rand_ind between 0 and 2253
index: 1807
total 4556
getting rand_ind between 0 and 2253
index: 461
total 4558
getting rand_ind between 0 and 2253
index: 1155
total 4560
done with image #569 

getting rand_ind between 0 and 2253
index: 1399
total 4562
getting rand_ind between 0 and 2253
index: 261
total 4564
getting rand_ind between 0 and 2253
index: 86
total 4566
getting rand_ind between 0 and 2253
index: 1277
total

index: 2206
total 6782
getting rand_ind between 0 and 2253
index: 1573
total 6784
done with image #847 

getting rand_ind between 0 and 2253
index: 354
total 6786
getting rand_ind between 0 and 2253
index: 239
total 6788
getting rand_ind between 0 and 2253
index: 25
total 6790
getting rand_ind between 0 and 2253
index: 75
total 6792
done with image #848 

getting rand_ind between 0 and 2253
index: 27
total 6794
getting rand_ind between 0 and 2253
index: 227
total 6796
getting rand_ind between 0 and 2253
index: 1682
total 6798
getting rand_ind between 0 and 2253
index: 947
total 6800
done with image #849 

getting rand_ind between 0 and 2253
index: 797
total 6802
getting rand_ind between 0 and 2253
index: 1232
total 6804
getting rand_ind between 0 and 2253
index: 154
total 6806
getting rand_ind between 0 and 2253
index: 472
total 6808
done with image #850 

getting rand_ind between 0 and 2253
index: 1575
total 6810
getting rand_ind between 0 and 2253
index: 1759
total 6812
getting rand_

index: 160
total 9004
getting rand_ind between 0 and 2253
index: 443
total 9006
getting rand_ind between 0 and 2253
index: 292
total 9008
done with image #1125 

getting rand_ind between 0 and 2253
index: 1489
total 9010
getting rand_ind between 0 and 2253
index: 712
total 9012
getting rand_ind between 0 and 2253
index: 1561
total 9014
getting rand_ind between 0 and 2253
index: 2161
total 9016
done with image #1126 

getting rand_ind between 0 and 2253
index: 1986
total 9018
getting rand_ind between 0 and 2253
index: 829
total 9020
getting rand_ind between 0 and 2253
index: 532
total 9022
getting rand_ind between 0 and 2253
index: 192
total 9024
done with image #1127 

getting rand_ind between 0 and 2253
index: 287
total 9026
getting rand_ind between 0 and 2253
index: 1583
total 9028
getting rand_ind between 0 and 2253
index: 786
total 9030
getting rand_ind between 0 and 2253
index: 559
total 9032
done with image #1128 

getting rand_ind between 0 and 2253
index: 1050
total 9034
gettin

index: 1413
total 11376
done with image #1421 

getting rand_ind between 0 and 2253
index: 445
total 11378
getting rand_ind between 0 and 2253
index: 1232
total 11380
getting rand_ind between 0 and 2253
index: 985
total 11382
getting rand_ind between 0 and 2253
index: 161
total 11384
done with image #1422 

getting rand_ind between 0 and 2253
index: 720
total 11386
getting rand_ind between 0 and 2253
index: 1500
total 11388
getting rand_ind between 0 and 2253
index: 1679
total 11390
getting rand_ind between 0 and 2253
index: 22
total 11392
done with image #1423 

getting rand_ind between 0 and 2253
index: 1152
total 11394
getting rand_ind between 0 and 2253
index: 147
total 11396
getting rand_ind between 0 and 2253
index: 2026
total 11398
getting rand_ind between 0 and 2253
index: 1759
total 11400
done with image #1424 

getting rand_ind between 0 and 2253
index: 1188
total 11402
getting rand_ind between 0 and 2253
index: 627
total 11404
getting rand_ind between 0 and 2253
index: 1965


index: 1125
total 13550
getting rand_ind between 0 and 2253
index: 1056
total 13552
done with image #1693 

getting rand_ind between 0 and 2253
index: 1280
total 13554
getting rand_ind between 0 and 2253
index: 1925
total 13556
getting rand_ind between 0 and 2253
index: 747
total 13558
getting rand_ind between 0 and 2253
index: 776
total 13560
done with image #1694 

getting rand_ind between 0 and 2253
index: 480
total 13562
getting rand_ind between 0 and 2253
index: 1012
total 13564
getting rand_ind between 0 and 2253
index: 1471
total 13566
getting rand_ind between 0 and 2253
index: 187
total 13568
done with image #1695 

getting rand_ind between 0 and 2253
index: 1645
total 13570
getting rand_ind between 0 and 2253
index: 1817
total 13572
getting rand_ind between 0 and 2253
index: 391
total 13574
getting rand_ind between 0 and 2253
index: 2137
total 13576
done with image #1696 

getting rand_ind between 0 and 2253
index: 727
total 13578
getting rand_ind between 0 and 2253
index: 175

index: 1377
total 15858
getting rand_ind between 0 and 2253
index: 571
total 15860
getting rand_ind between 0 and 2253
index: 1904
total 15862
getting rand_ind between 0 and 2253
index: 1486
total 15864
done with image #1982 

getting rand_ind between 0 and 2253
index: 216
total 15866
getting rand_ind between 0 and 2253
index: 1450
total 15868
getting rand_ind between 0 and 2253
index: 2091
total 15870
getting rand_ind between 0 and 2253
index: 333
total 15872
done with image #1983 

getting rand_ind between 0 and 2253
index: 200
total 15874
getting rand_ind between 0 and 2253
index: 524
total 15876
getting rand_ind between 0 and 2253
index: 1110
total 15878
getting rand_ind between 0 and 2253
index: 921
total 15880
done with image #1984 

getting rand_ind between 0 and 2253
index: 1796
total 15882
getting rand_ind between 0 and 2253
index: 160
total 15884
getting rand_ind between 0 and 2253
index: 2032
total 15886
getting rand_ind between 0 and 2253
index: 2019
total 15888
done with i

In [82]:
print(len(x),len(y))

18024 18024


In [112]:
def train_test_val(x,y): # make train & test, then split val from train
    x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state=1)
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state=1)
    print(len(x_train),len(x_test),len(x_val)) # acceptable numbers for this dataset
    return x_train, y_train, x_test, y_test, x_val, y_val

In [113]:
x_train,y_train, x_test,y_test, x_val,y_val = train_test_val(x,y)

12977 3605 1442


In [50]:
# x_train[0]

In [114]:
print(np.shape(x_train)) # check shapes
print(np.shape(y_train))
print(np.shape(x_test))
print(np.shape(y_test))
print(np.shape(x_val))
print(np.shape(y_val)) # no ghost tensor issues this time around, good

(12977, 2, 200, 200, 3)
(12977,)
(3605, 2, 200, 200, 3)
(3605,)
(1442, 2, 200, 200, 3)
(1442,)


## network building

In [56]:
K.clear_session() # wipe GPU memory

In [57]:
# baby's first convolutional network
def create_base_network(input_shape):
    inp = Input(shape=input_shape) # consider size of filters
    x = Conv2D(64,(10,10), activation='relu',input_shape=input_shape) 
    x = MaxPooling2D() # reassigning x 'stacks' layers
    x = Conv2D(128,(7,7),activation='relu', # i'm reluing on this activation a bit much
              kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))
    x = MaxPooling2D()
    x = Conv2D(128,(4,4),activation='relu',
              kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))
    x = MaxPooling2D()
    x = Conv2D(128,(4,4),activation='relu',
              kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))
    x = MaxPooling2D() # 512 kills runtime
    x = Conv2D(256,(4,4),activation='relu',
              kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))
    x = Flatten()(inp)
    x = Dense(4096, activation='relu',
             kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))(x)
    return Model(inp, x) 

input_shape = np.shape(x_train[0][0])
base_network = create_base_network(input_shape)

In [84]:
# the magic
input_a = Input(shape=input_shape) # create input feeds
input_b = Input(shape=input_shape)

processed_a = base_network(input_a) # run left/right inputshapes through network
processed_b = base_network(input_b) 

distance = Lambda(euclidean_distance, # define distance metric
                      output_shape=eucl_dist_output_shape)([processed_a, processed_b])
    
model = Model([input_a, input_b], distance) # keras API model for input, output

In [85]:
# configurations
img_gen = ImageDataGenerator()
patience_=3 # play around with this
es = EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=patience_,verbose=1,restore_best_weights=True)
calb = []
calb.append(es) # early stopping to callbacks

In [86]:
# generator
def generator(all_pairs, labels, aug_gen_obj, batch_size=16):
    '''is there a reason for a specific batch_size? not really. tradeoffs in speed vs. efficiency
    i'm sampling randomly from all pairs, so the data is shuffled'''
    while True:
        batch_faces = sample(all_pairs, batch_size) # get a batch of face pairs [face1, face2] w/o replacement
        faces_left, faces_right = [], [] # outputs. holds pairs
        labels = np.ones(batch_size) # label vector output. length matches number of pairs in batch
        for c, face_pair in enumerate(batch_faces): # [left_face, right_face]
            if np.array_equal(face_pair[0], face_pair[1]): # it's the same person
                face_pair[1] = random_aug(face_pair[1], aug_gen_obj) # randomly augment the right_face
                labels[c] = 1 # set corresponding label to True
            else: # it's a different person
                labels[c] = 0
            faces_left.append(face_pair[0])
            faces_right.append(face_pair[1])
        yield [faces_left, faces_right], labels

## train network

In [115]:
model.compile(loss=contrastive_loss, 
              optimizer = optimizers.Adam(0.00001), metrics=['acc'])
history = model.fit_generator(generator(x_train,y_train,img_gen),
                validation_data = generator(x_val,y_val,img_gen),
                epochs=10, verbose=1, steps_per_epoch=100, 
                validation_steps=10, callbacks=calb)

Epoch 1/10
100/100 [==============================] - 359s 4s/step - loss: 14350.5005 - acc: 0.4944 - val_loss: 4277.6775 - val_acc: 0.4812
Epoch 2/10
100/100 [==============================] - 362s 4s/step - loss: 5856.0816 - acc: 0.4931 - val_loss: 8175.5046 - val_acc: 0.5938
Epoch 3/10
100/100 [==============================] - 364s 4s/step - loss: 6849.2734 - acc: 0.5550 - val_loss: 651.4252 - val_acc: 0.5813
Epoch 4/10
100/100 [==============================] - 364s 4s/step - loss: 3406.3065 - acc: 0.5756 - val_loss: 3462.1249 - val_acc: 0.6187
Epoch 5/10
100/100 [==============================] - 372s 4s/step - loss: 15981.6492 - acc: 0.6131 - val_loss: 48664.0909 - val_acc: 0.5813
Epoch 6/10
100/100 [==============================] - 364s 4s/step - loss: 17815.3757 - acc: 0.6125 - val_loss: 33243.7418 - val_acc: 0.7188
Epoch 7/10
100/100 [==============================] - 366s 4s/step - loss: 3847.8189 - acc: 0.6619 - val_loss: 506.4089 - val_acc: 0.7812
Epoch 8/10
100/100 [====

## initial run complete
- 72% acc in 1 hour and 10 epochs. 
    - neat, but i'm certainly still limited by the data itself (transformations compensating for lack of multi-shot cases) and complexity (running this on a macbook).
    - at least i've got the process modular & simplified now, so I can build a bigger model to run in SageMaker

ah, i closed the notebook last night without saving the model.
 considering i'm not going to get any decent testing results from it, no worries
- tasks:
    - learn SageMaker, then double the nodes in all layers and run it
    - figure out how to "chop off" the top layer to get the "certainty" decimal to cast as similarity
    - salvage old image_comparison code into something workable
   

In [ ]:
# don't know what those shape dimension expected mismatch errors were - didn't get them last time i ran this framework
# when i ran the model.compile again, it gave me a different size mismatch
# ran it once more, it's training with no issues
# ah, a minute later

## exorcism
- i finally resolved the "ghost tensor" issue i'd been having since the first run-through
    - i accidentally slipped the high res images in with the training batch, so it gave me shape mismatch on compile
    - but i also found some pairs that were shape (2,) instead of (2,200,200,3)
        - I added cv2.resize() to the image_to_array method and that fixed both issues at once
            - not sure what the takeaways are here. i sort of stumbled across the correct answer. is it intuitive that resizing an image array will fix the "full shape not detected" error? not to me, at least